In [2]:
import csv
import os
import random
from autogluon.tabular import TabularDataset, TabularPredictor
from os.path import exists
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
import pickle
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import numpy as np
import json
from statistics import mean

In [72]:
# define properties map to determine which qnodes and properties you want to predict the object for
properties_map = {"P1412":{"Q7747","Q11116"},"P106":{"Q1701254","Q5608","Q207","Q36153","Q44437","Q15935","Q36159","Q762"},"P19":{"Q180589"},"P102":{"Q22686"},"P641":{"Q44437"},"P180":{"Q12418"},"P136":{"Q175036","Q506418"},"P186":{"Q18891156"}}
#properties_map = {"P27":{"Q1426"}}
print(properties_map)
print(len(properties_map))

{'P27': {'Q1426'}}
1


In [5]:
property_label="P27"
label = 'Yval'
percent_train_data = .75

In [73]:
# Read input file and create subject/object map
all_node_maps = {}
for property_label in properties_map:
    all_lines = []
    node_map = {}
    print("Reading claims data for property",property_label,"...(may take some time depending on chosen property)")
    with open(f"../data/propertiesSplit_final/claims.{property_label}.tsv", encoding="utf-8") as in_file:
        tsv_reader = csv.reader(in_file, delimiter="\t")
        header = next(tsv_reader)
        for line in tsv_reader:
            all_lines.append(line)
            if line[0] in node_map:
                node_map[line[0]].append(line[2])
            else:
                node_map[line[0]] = [line[2]]
    print(property_label,"claims:",len(all_lines))
    print("Found",len(node_map),"distinct nodes with at least one",property_label,"relationship")
    all_node_maps[property_label] = node_map
print(len(all_node_maps))

Reading claims data for property P27 ...(may take some time depending on chosen property)
P27 claims: 3975380
Found 3678428 distinct nodes with at least one P27 relationship
1


In [89]:
# create shuffled output file
for property_label in properties_map:
    print("Creating shuffled output file...")
    random.shuffle(all_lines)
    with open(f"../data/propertiesSplit_final/claims.{property_label}.shuffled.tsv", "w", encoding="utf-8", newline='') as out_file:
        tsv_writer = csv.writer(out_file, delimiter="\t")
        tsv_writer.writerow(header)
        for line in all_lines:
            tsv_writer.writerow(line)
    print("Shuffled output written to claims."+property_label+".tsv")

Creating shuffled output file...
Shuffled output written to claims.P828.tsv


In [75]:
# create embeddings-based training data
# nodes_to_check=["Q16"]
transEheader = ['Qnode']
for i in range(0,100):
    transEheader.append("pos"+str(i))
transEheader.append("Yval")
file_map = {}
for property_label in properties_map:
    file1 = open(f"../data/link_prediction_data/{property_label}.transE.tsv", "w", encoding="utf-8", newline='')
    file2 = open(f"../data/temp/custom_test_{property_label}.tsv", 'w', encoding="utf-8", newline='')
    tsv_writer = csv.writer(file1, delimiter="\t")
    temp_writer = csv.writer(file2, delimiter="\t")
    tsv_writer.writerow(transEheader)
    temp_writer.writerow(transEheader)
    file_map[property_label] = [tsv_writer, temp_writer, file1, file2]

# if (exists(f"../data/link_prediction_data/{property_label}.transE.tsv")):
#     print(f"File ../data/link_prediction_data/{property_label}.transE.tsv already exists!")
#else:
print("Reading the transE file...(this will take some time)")
with open(f"../embeddings/profile_graph_embeddings.transE.tsv", encoding="utf-8") as in_file:
    tsv_reader = csv.reader(in_file, delimiter=" ")
    next(tsv_reader)
    for line in tsv_reader:
        for property_label in properties_map:
            if line[0] in properties_map[property_label]:
                file_map[property_label][1].writerow(line)
                print(property_label)
            elif line[0] in all_node_maps[property_label]:
                this_arr = []
                for elem in line:
                    this_arr.append(elem)
                randchoice = random.choice(all_node_maps[property_label][line[0]])
                this_arr.append(randchoice)
                file_map[property_label][0].writerow(this_arr)

for property_label in file_map:
    file_map[property_label][2].close()
    file_map[property_label][3].close()

print("Created training data file for property",property_label)

Reading the transE file...(this will take some time)
P27
Created training data file for property P27


In [207]:
# create embeddings-based training data for a single unit type
nodes_to_check=["Q3895"]
if (exists(f"../data/link_prediction_data/{property_label}.transE.tsv")):
    print(f"File ../data/link_prediction_data/{property_label}.transE.tsv already exists!")
else:
    transEheader = ['Qnode']
    for i in range(0,100):
        transEheader.append("pos"+str(i))
    transEheader.append("Yval")
    linecount = 0
    print("Reading the transE file...(this will take some time)")
    with open(f"../embeddings/profile_graph_embeddings.transE.tsv", encoding="utf-8") as in_file, open(f"../data/link_prediction_data/{property_label}.transE.tsv", "w", encoding="utf-8", newline='') as out_file, open(f"../data/temp/custom_test_{property_label}.tsv", 'w', encoding="utf-8", newline='') as temp_file:
        tsv_reader = csv.reader(in_file, delimiter=" ")
        tsv_writer = csv.writer(out_file, delimiter="\t")
        temp_writer = csv.writer(temp_file, delimiter="\t")
        next(tsv_reader)
        tsv_writer.writerow(transEheader)
        temp_writer.writerow(transEheader)
        for line in tsv_reader:
            if line[0] in nodes_to_check:
                temp_writer.writerow(line)
            elif line[0] in node_map:
                this_arr = []
                for elem in line:
                    this_arr.append(elem)
                #randchoice = random.choice(node_map[line[0]])
                for elem in random.sample(node_map[line[0]], len(node_map[line[0]])):
                    unit = elem.split("Q")
                    if len(unit) > 1:
                        if unit[1] == "4917":
                            asnum = elem.replace("+","").split("[")[0].split("Q")[0]
                            assert asnum.replace(".","").replace("-","").isnumeric(), asnum
                            this_arr.append(asnum)
                            tsv_writer.writerow(this_arr)
                            linecount += 1
                            break

    print("Created training data file for property",property_label,"with",linecount,"rows")

Reading the transE file...(this will take some time)
Created training data file for property P2139 with 9390 rows


In [76]:
# load up the training/test data
all_data = TabularDataset(f"../data/link_prediction_data/{property_label}.transE.tsv")

In [77]:
# make train/test split
print("Making train/test split for property",property_label,"with",str(percent_train_data)[2:],"% training data...")
# limit data to some number of training rows
short_data=all_data
short_data = short_data
print("Data length:",int(len(short_data)))
train_data = short_data[:int(len(short_data)*percent_train_data)]
test_data = short_data[int(len(short_data)*percent_train_data):]
print("Training data size:",len(train_data))
print("Test data size:",len(test_data))

Making train/test split for property P27 with 75 % training data...
Data length: 3381439
Training data size: 2536079
Test data size: 845360


In [78]:
# fit autogluon models
#save_path = f'../embeddings/models/agModels.{property_label}.transE'
save_path = f'../embeddings/models/KNNModels.{property_label}.transE'
print("Summary of class variable: \n", train_data[label].describe())
#predictor = TabularPredictor(label=label, path=save_path, learner_kwargs={"label_count_threshold":10}).fit(train_data, hyperparameters={'NN':{},'GBM':{},'XT':{},'KNN':{}})
#print("Best model:",predictor.get_model_best())
#print(f"Model saved to ../embeddings/models/agModels.{property_label}.transE")
neigh = KNeighborsClassifier(n_neighbors=25)
#neigh = KNeighborsRegressor(n_neighbors=25)
train_dropped = train_data.drop(columns=["Qnode","Yval"])
y_val = train_data["Yval"]
predictor = neigh.fit(train_dropped, y_val)
pickle.dump(predictor, open(save_path, 'wb'))

Summary of class variable: 
 count     2536079
unique       2443
top           Q30
freq       320762
Name: Yval, dtype: object


In [68]:
# run the 2 lines below to load the saved model; if the above cell has just been ran this is not necessary
save_path = f'../embeddings/models/KNNModels.{property_label}.transE'
# predictor = TabularPredictor.load(save_path)
predictor = pickle.load(open(save_path, 'rb'))

In [ ]:
print("Making predictions over test data...")
y_test = test_data[label]
test_data_nolab = test_data.drop(columns=[label])
y_pred = list(predictor.predict(test_data_nolab))
print("Predictions saved in y_pred variable")

In [ ]:
entries = list(test_data["Qnode"])
correct = 0
total = 0
for index, entry in enumerate(entries):
    prediction = y_pred[index]
    if prediction in node_map[entry]:
        correct+=1
    total+=1
print("Overall accuracy:",(correct/total))

In [10]:
numeric_property_labels = ["P1971","P2067","P2139","P2250","P2927","P3001"]
property_labels = ["P27","P102","P106","P136","P180","P186","P641","P1412"]
all_properties = ["P27","P102","P106","P136","P180","P186","P641","P1412","P1971","P2067","P2139","P2250","P2927","P3001"]

for property_label in numeric_property_labels:
    nodes_to_check = []
    print(property_label)
    
    all_data = TabularDataset(f"../data/link_prediction_data/{property_label}.transE.tsv")
    print("Data length:",int(len(all_data)))
    train_data = all_data[:int(len(all_data)*percent_train_data)]
    y_val = train_data["Yval"]
    
    save_path = f'../embeddings/models/KNNModels.{property_label}.transE'
    predictor = pickle.load(open(save_path, 'rb'))
    
    with open(f"../data/temp/custom_test_{property_label}.tsv", encoding="utf-8") as in_file:
            tsv_reader = csv.reader(in_file, delimiter="\t")
            next(tsv_reader)
            for line in tsv_reader:
                nodes_to_check.append(line[0])

    results = {}
    for node in nodes_to_check:
        results[node] = {}

    y_val_ordered = sorted(set(y_val))

    preddata = TabularDataset(f"../data/temp/custom_test_{property_label}.tsv").drop(columns=[label,"Qnode"])
    #probabilities = predictor.predict_proba(preddata)
    probabilities = predictor.predict(preddata)

    for ind, node in enumerate(nodes_to_check):
        with open(f"../../surprising-facts/distance_results/profile.transE.supervised.lhs/results_{node}_{property_label}.tsv", "w", encoding="utf-8", newline='') as out_file:
            tsv_writer = csv.writer(out_file, delimiter="\t")
            index = ind
            res_map = {}
            for in2, val in enumerate(probabilities[index]):
                if val > 0:
                    res_map[y_val_ordered[in2]] = 1-val
            print(node)
            print({k: v for k, v in sorted(res_map.items(), key=lambda item: item[1], reverse=True)})
            for key in res_map:
                tsv_writer.writerow([key, res_map[key]]) 

P27
Data length: 3381439
Q1426
{'Q115': 0.96, 'Q32': 0.96, 'Q801': 0.96, 'Q974': 0.96, 'Q30': 0.92, 'Q191': 0.84, 'Q40': 0.76, 'Q39': 0.64}
P102
Data length: 350634
Q22686
{'Q849158': 0.96, 'Q29552': 0.56, 'Q29468': 0.48}
P106
Data length: 5852159
Q15935
{'Q10800557': 0.96, 'Q1259917': 0.96, 'Q1327329': 0.96, 'Q158852': 0.96, 'Q2340668': 0.96, 'Q33999': 0.96, 'Q3427922': 0.96, 'Q557880': 0.96, 'Q5716684': 0.96, 'Q855091': 0.96, 'Q183945': 0.92, 'Q55960555': 0.92, 'Q639669': 0.92, 'Q36834': 0.84, 'Q488205': 0.8}
Q5608
{'Q10800557': 0.96, 'Q183945': 0.96, 'Q18814623': 0.96, 'Q2526255': 0.96, 'Q28389': 0.96, 'Q3282637': 0.96, 'Q37226': 0.96, 'Q386854': 0.96, 'Q4351403': 0.96, 'Q488205': 0.96, 'Q49757': 0.96, 'Q55960555': 0.96, 'Q6168364': 0.96, 'Q753110': 0.96, 'Q765778': 0.96, 'Q36180': 0.92, 'Q639669': 0.92, 'Q177220': 0.88, 'Q36834': 0.88}
Q762
{'Q121594': 0.96, 'Q1234713': 0.96, 'Q14623005': 0.96, 'Q1622272': 0.96, 'Q170790': 0.96, 'Q177220': 0.96, 'Q205375': 0.96, 'Q214917': 0.96, 'Q

AttributeError: 'KNeighborsRegressor' object has no attribute 'predict_proba'

In [2]:
# read second benchmark facts
import csv
count = 0
properties_map = {}
all_answers = set()
with open(f"../wd_bm_2.csv") as in_file:
    csv_reader = csv.reader(in_file, delimiter=",")
    next(csv_reader)
    for line in csv_reader:
        if line[3] != "":
            split = "|".join(line[3].split(",")).split("|")
            for fact in split:
                if "cent" not in fact and "_" not in fact and fact != "":
                    factsplit = fact.split("=")
                    count += 1
                    if factsplit[0] in properties_map:
                        properties_map[factsplit[0]].add(line[1])
                    else:
                        properties_map[factsplit[0]] = {line[1]}
                    all_answers.add(factsplit[1])
print(all_answers)

{'Q13381572', 'Q6581097', 'Q37073', 'Q932650', 'Q860450', 'Q49085', 'Q3282637', 'Q22662561', 'Q1111310', 'Q2526255', 'Q36834', 'Q177220', 'Q11338576', 'Q45', 'Q50001', 'Q13474373', 'Q8341', 'Q10798782', 'Q989442', 'Q43845', 'Q183387', 'Q15295720', 'Q104238320', 'Q3724813', 'Q12718299', 'Q1320315', 'Q28389', 'Q2405480', 'Q11296761', 'P30', 'Q15982795', 'Q145', 'Q33999', 'Q4610556', 'Q11696', 'Q34', 'Q23845879', 'Q19204627', 'Q67175841', 'Q865851', 'Q5716684', 'Q16947675', 'Q16', 'Q668', 'Q82955', 'Q2742632', 'Q45981', 'Q327591', 'Q655286', 'Q169470', 'Q6581072', 'Q465501', 'Q1371091', 'Q41323', 'Q40', 'Q10497074', 'Q1414443', 'Q488205', 'Q9268', 'Q519076', 'Q30', 'Q11607585', 'Q11124885', 'Q60', 'Q15966504', 'Q27'}


In [7]:
# scan prediction files and add embedding results
properties = {}

file_map = {}
properties_map = ["P1971", "P2067", "P2139", "P2250", "P2927", "P3001"]
for property_label in properties_map:
    file1 = open(f"../data/results/bm1/embeddings/bm1_results.{property_label}.embeddings.tsv", "w", encoding="utf-8", newline='')
    tsv_writer = csv.writer(file1, delimiter="\t")
    file_map[property_label] = [tsv_writer, file1]
    properties[property_label] = set()
    with open(f"../data/results/bm1/bm1_results.{property_label}.tsv") as in_file:
        tsv_reader = csv.reader(in_file, delimiter="\t")
        for line in tsv_reader:
            properties[property_label].add(line[2])
print(properties)
    
with open(f"../embeddings/profile_graph_embeddings.TransE.tsv") as transE:
    tsv_reader = csv.reader(transE, delimiter=" ")
    header = next(tsv_reader)
    for line in tsv_reader:
        for property_label in properties:
            for cla in properties[property_label]:
                if line[0] == cla:
                    print(property_label, cla)
                    file_map[property_label][0].writerow(line)

for property_label in properties_map:
    file_map[property_label][1].close()

{'P1971': {'Q5_P1971_3.0-4.0__'}, 'P2067': {'Q5_P2067_210.0-240.0__Q100995'}, 'P2139': {'Q4830453_P2139_6764000000.0-217267000000.0__Q4916'}, 'P2250': {'Q3624078_P2250_73.861-76.577__Q577'}, 'P2927': {'Q3624078_P2927_2.6-5.7__Q11229'}, 'P3001': {'Q3624078_P3001_62.5-65.0__Q24564698'}}
P2250 Q3624078_P2250_73.861-76.577__Q577
P2927 Q3624078_P2927_2.6-5.7__Q11229
P2139 Q4830453_P2139_6764000000.0-217267000000.0__Q4916
P3001 Q3624078_P3001_62.5-65.0__Q24564698
P2067 Q5_P2067_210.0-240.0__Q100995
P1971 Q5_P1971_3.0-4.0__


In [1]:
import os
properties = {}

#all_answers = ["Q1860","Q188","Q7737","Q9027","Q1321","Q10800557","Q36180","Q10798782","Q947873","Q3282637","Q258","Q39","Q884","Q142","Q30","Q60","Q90","Q220","Q1055","Q84","Q482980","Q3282637","Q131524","Q488205","Q1028181","Q14089670","Q82955","Q189290","Q1028181","Q177220","Q4610556","Q2405480","Q131524","Q488205","Q5716684","Q29552","Q29468","Q79854","Q7320","Q49768","Q639669","Q2405480","Q36180","Q82955","Q10873124","Q41323","Q5369","Q838089","Q32112","Q5378","Q42973","Q2526255","Q82955","Q177220","Q2309784","Q10798782","Q3665646","Q28389","Q1650915","Q36180","Q193391","Q350979","Q593644","Q1028181","Q81096","Q9027","Q1860","Q7737","Q150","Q188","Q12280","Q8502","Q527","Q107425","Q345","Q742333","Q134307","Q1935974","Q191163","Q192110","Q389782","Q175166","Q296955","Q11472","Q12321255","Q4259259","Q52207399","Q860626","Q157394","Q130232","Q842256","Q5_P1412_Q1860","Q5_P1412_Q188","Q5_P1412_Q7737","Q5_P1412_Q9027","Q5_P1412_Q1321","Q5_P106_Q10800557","Q5_P106_Q36180","Q5_P106_Q10798782","Q5_P106_Q947873","Q5_P106_Q3282637","Q5_P27_Q258","Q5_P27_Q39","Q5_P27_Q884","Q5_P27_Q142","Q5_P27_Q30","Q5_P1971_0.0-2.0__","Q5_P1971_2.0-3.0__","Q5_P1971_3.0-4.0__","Q5_P1971_4.0-210.0__","Q5_P19_Q60","Q5_P19_Q90","Q5_P19_Q220","Q5_P19_Q1055","Q5_P19_Q84","Q5_P106_Q482980","Q5_P106_Q3282637","Q5_P106_Q131524","Q5_P106_Q488205","Q5_P106_Q1028181","Q5_P106_Q14089670","Q5_P106_Q82955","Q5_P106_Q189290","Q5_P106_Q1028181","Q5_P106_Q177220","Q5_P106_Q4610556","Q5_P106_Q2405480","Q5_P106_Q131524","Q5_P106_Q488205","Q5_P106_Q5716684","Q5_P2067_1.0-181.0__Q100995","Q5_P2067_181.0-195.0__Q100995","Q5_P2067_195.0-210.0__Q100995","Q5_P2067_210.0-240.0__Q100995","Q5_P2067_240.0-555.0__Q100995","Q5_P102_Q29552","Q5_P102_Q29468","Q5_P102_Q79854","Q5_P102_Q7320","Q5_P102_Q49768","Q5_P106_Q639669","Q5_P106_Q2405480","Q5_P106_Q36180","Q5_P106_Q82955","Q5_P106_Q10873124","Q5_P641_Q41323","Q5_P641_Q5369","Q5_P641_Q838089","Q5_P641_Q32112","Q5_P641_Q5378","Q5_P106_Q42973","Q5_P106_Q2526255","Q5_P106_Q82955","Q5_P106_Q177220","Q5_P106_Q2309784","Q5_P106_Q10798782","Q5_P106_Q3665646","Q5_P106_Q28389","Q5_P106_Q1650915","Q5_P106_Q36180","Q5_P106_Q193391","Q5_P106_Q350979","Q5_P106_Q593644","Q5_P106_Q1028181","Q5_P106_Q81096","Q5_P1412_Q9027","Q5_P1412_Q1860","Q5_P1412_Q7737","Q5_P1412_Q150","Q5_P1412_Q188","Q3624078_P2250_76.59756-85.41707__Q577","Q3624078_P2250_63.33-69.862__Q577","Q3624078_P2250_70.197-73.82683__Q577","Q3624078_P2250_51.835-63.238__Q577","Q3624078_P2250_73.861-76.577__Q577","Q3624078_P3001_55.0-60.0__Q24564698","Q3624078_P3001_62.5-65.0__Q24564698","Q3624078_P3001_60.0-62.5__Q24564698","Q3624078_P3001_65.25-67.0__Q24564698","Q3624078_P2927_8.4-27.9__Q11229","Q3624078_P2927_1.5-2.5__Q11229","Q3624078_P2927_0.0-0.2__Q11229","Q3624078_P2927_2.6-5.7__Q11229","Q3624078_P2927_0.3-1.444__Q11229","Q3305213_P180_Q12280","Q3305213_P180_Q8502","Q3305213_P180_Q527","Q3305213_P180_Q107425","Q3305213_P180_Q345","Q3305213_P136_Q742333","Q3305213_P136_Q134307","Q3305213_P136_Q1935974","Q3305213_P136_Q191163","Q3305213_P136_Q192110","Q3305213_P186_Q389782","Q3305213_P186_Q175166","Q3305213_P186_Q296955","Q3305213_P186_Q11472","Q3305213_P186_Q12321255","Q3305213_P186_Q4259259","Q11424_P136_Q52207399","Q11424_P136_Q860626","Q11424_P136_Q157394","Q11424_P136_Q130232","Q11424_P136_Q842256","Q4830453_P2139_1.0-108589000.0__Q4916","Q4830453_P2139_438000000.0-1590000000.0__Q4916","Q4830453_P2139_113000000.0-427800000.0__Q4916","Q4830453_P2139_6764000000.0-217267000000.0__Q4916","Q4830453_P2139_1610000000.0-6745000000.0__Q4916"]
all_answers = ["Q34174","Q2982063","Q66650486","Q14955281","Q88400354"]
answerfile = open(f"../embeddings/bm1.answers.embeddings.custom.tsv", "w", encoding="utf-8", newline='')
tsv_writer = csv.writer(answerfile, delimiter="\t")

with open(f"../embeddings/profile_graph_embeddings.TransE.tsv") as transE:
    tsv_reader = csv.reader(transE, delimiter=" ")
    header = next(tsv_reader)
    for line in tsv_reader:
        if line[0] in all_answers:  
            tsv_writer.writerow(line)
    
answerfile.close()

NameError: name 'csv' is not defined

In [47]:
# get MRR score from distance files

with open("../wd_json_trivia.qnodes.json", 'r') as f:
    questions = json.load(f)
    
mrr_scores = []

for i, q in enumerate(questions):
    for answer in q["answers"]:
        ans_fact_ids = []
        for qnode in answer["qnodes"]:
            fact_id = "{}_{}_{}".format(q["class"], q["property"], qnode)
            if q["wd_units"] is not None:
                fact_id += "__" + q["wd_units"]
            ans_fact_ids.append(fact_id)
        answer["fact_ids"] = ans_fact_ids
        
# assign outliers
for q in questions:
    surprise_score = []
    for ans in q["answers"]:
        surprise_score.append(ans["gt_surprise"])
    outlier_scores = sorted(surprise_score, reverse=True)[:2]
    for ans in q["answers"]:
        if ans["gt_surprise"] in outlier_scores:
            ans["outlier"] = True
        else:
            ans["outlier"] = False
        
for q in questions:
    print(q["lexicalized"])
    keystring = "qnodes"
    if q["is_numeric_answer"]:
        keystring = "fact_ids"
    src_ent = q["entity"]
    edge = q["property"]

    targets = {}
    for ans in q["answers"]:
        if ans["truth_value_fact_checked"] == True:
            if ans["outlier"] == True:
                for qnode in ans[keystring]:
                    targets[qnode] = [0,0]

    if len(targets) > 0:
        file = f"../../surprising-facts/distance_results/profile.transE.supervised.lhs/results_{src_ent}_{edge}.tsv"
        count = 0
        with open(file, "r") as in_file:
            reader = csv.reader(in_file, delimiter="\t")
            for row in reader:
                if row[0] in targets:
                    targets[row[0]][0] = float(row[1])
                    targets[row[0]][1] += 1
        with open(file, "r") as in_file:
            reader = csv.reader(in_file, delimiter="\t")
            for row in reader:
                dist = float(row[1])
                for tar in targets:
                    if row[0] != tar:
                        if dist < targets[tar][0]:
                            targets[tar][1] += 1
        print(targets)
        for tar in targets:
            if targets[tar][1] == 0:
                mrr_scores.append(0)
            else:
                mrr_scores.append(1/targets[tar][1])

print(len(mrr_scores))
print("Final MRR:")
print(mean(mrr_scores))

What language(s) does Vladimir Putin speak, write, or sign with?
{'Q9027': [0, 0]}
What occupation(s) has John Oliver had?
{'Q10800557': [0.88, 1]}
Please select the country or countries in which Roger Federer possesses citizenship
How many children does Arnold Schwarzenegger have?
{'Q5_P1971_4.0-210.0__': [0, 0]}
What is Boris Johnson's place of birth?
What occupation(s) has Eminem had?
{'Q482980': [0, 0]}
What occupation(s) has George W. Bush had?
{'Q14089670': [0, 0]}
What occupation(s) has Beyoncé had?
{'Q4610556': [0, 0], 'Q2405480': [0, 0]}
What is Donald Trump's mass in pounds circa. 2019?
What political party/parties has Donald Trump been a member of?
What occupation(s) has John Cena had?
{'Q36180': [0.76, 1]}
What sport(s) has John Cena participated in?
{'Q41323': [0.6, 1]}
What occupation(s) has Kanye West had?
What occupation(s) has LeBron James had?
{'Q28389': [0, 0]}
What occupation(s) did Leonardo Da Vinci have?
{'Q350979': [0, 0]}
What language(s) did Ruth Bader Ginsburg